<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/timesformer/TimeSformer-huggingface-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 23.5 MB/s eta 0:00:00


In [ ]:
from ftplib import FTP
import io
import random

# FTP 서버 정보
ftp_server = "121.136.96.223"  # Synology NAS의 IP 주소
ftp_port = 21  # 기본 포트 21, 다른 포트를 사용하는 경우 해당 포트 번호 입력
ftp_user = "donghuna_ftp"  # NAS 로그인 사용자명
ftp_password = "Dlehdgns0892!@!?n"  # NAS 로그인 비밀번호
# file_path = "homes/donghuna/database/Diving48_rgb/rgb/_8Vy3dlHg2w_00000.mp4"
# local_file_path = "eating_spaghetti.mp4"  # 로컬에 저장할 파일 이름
folder_path = "homes/donghuna/database/Diving48_rgb/rgb/"  # 동영상 파일이 있는 폴더 경로


# FTP 연결 설정
ftp = FTP()
ftp.connect(ftp_server, ftp_port)
ftp.login(user=ftp_user, passwd=ftp_password)

# 액티브 모드 설정 (패시브 모드 비활성화)
ftp.set_pasv(True)

# 파일 다운로드
# with open(local_file_path, 'wb') as local_file:
#     ftp.retrbinary(f'RETR {file_path}', local_file.write)

# 동영상을 메모리에 저장
# video_data = io.BytesIO()
# ftp.retrbinary(f'RETR {file_path}', video_data.write)
# video_data.seek(0)

# 연결 종료
# ftp.quit()

# print(f"File downloaded successfully to {local_file_path}")

# 폴더 내 동영상 파일 목록 가져오기
ftp.cwd(folder_path)
file_list = ftp.nlst()  # 폴더 내 파일 목록

# 동영상 파일 필터링 (mp4 확장자)
video_files = [file for file in file_list if file.endswith('.mp4')]

# 임의로 10개 파일 선택
random_video_files = random.sample(video_files, min(10, len(video_files)))



In [ ]:
import av # 동영상 파일을 처리하기 위한 라이브러리.
import torch # PyTorch 라이브러리로, 모델 학습 및 추론에 사용.
import numpy as np # 배열 및 수학적 연산을 위한 라이브러리.

from transformers import AutoImageProcessor, TimesformerForVideoClassification # Hugging Face Transformers 라이브러리로, 여기서는 Timesformer 모델을 로드하고 사용.
from huggingface_hub import hf_hub_download # Hugging Face Hub에서 데이터를 다운로드하기 위한 함수.

np.random.seed(0)

# PyAV를 사용하여 지정된 인덱스의 프레임을 읽어오는 함수입니다.
# container는 PyAV의 동영상 컨테이너 객체이고, indices는 읽어올 프레임 인덱스 리스트입니다.
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

# 이미지 전처리 및 모델 로드
image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

# 선택된 파일들 처리
for file_name in random_video_files:
    print(f"Processing {file_name}...")

    # 동영상을 메모리에 저장
    video_data = io.BytesIO()
    ftp.retrbinary(f'RETR {file_name}', video_data.write)
    video_data.seek(0)

    # PyAV를 사용하여 메모리에서 동영상 읽기
    container = av.open(video_data, format='mp4')

    # 프레임 샘플링
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    # 모델 입력 준비
    inputs = image_processor(list(video), return_tensors="pt")

    # 모델 추론
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 예측 결과 확인
    predicted_label = logits.argmax(-1).item()
    print(f"Predicted label for {file_name}: {model.config.id2label[predicted_label]}")

# 연결 종료
ftp.quit()

Processing yHo2oERGo7A_00037.mp4...


/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)


Predicted label for yHo2oERGo7A_00037.mp4: springboard diving
Processing fohMq9tOn6E_00010.mp4...
Predicted label for fohMq9tOn6E_00010.mp4: springboard diving
Processing Q--70-ELqLw_00162.mp4...
Predicted label for Q--70-ELqLw_00162.mp4: springboard diving
Processing Y7QZcr24ye0_01024.mp4...
Predicted label for Y7QZcr24ye0_01024.mp4: springboard diving
Processing nOlRwoxsDJ0_00080.mp4...
Predicted label for nOlRwoxsDJ0_00080.mp4: swimming backstroke
Processing D6zILEKIJbk_00022.mp4...
Predicted label for D6zILEKIJbk_00022.mp4: springboard diving
Processing zBXtMcI41z8_00007.mp4...
Predicted label for zBXtMcI41z8_00007.mp4: pull ups
Processing D8YKHC5hmUs_00217.mp4...
Predicted label for D8YKHC5hmUs_00217.mp4: springboard diving
Processing Y4ARBzok9aU_00024.mp4...
Predicted label for Y4ARBzok9aU_00024.mp4: springboard diving
Processing c7FHotPDZIw_00096.mp4...
Predicted label for c7FHotPDZIw_00096.mp4: trapezing


'221 Goodbye. You uploaded 0 bytes and downloaded 5710.10 KB.'

In [3]:
import json
import os
import random
from ftplib import FTP
import io
import numpy as np
import av
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import TimesformerForVideoClassification, get_linear_schedule_with_warmup, AdamW

# FTP 서버 정보
ftp_server = "121.136.96.223"
ftp_port = 21
ftp_user = "donghuna_ftp"
ftp_password = "Dlehdgns0892!@!?n"
folder_path = "homes/donghuna/database/Diving48_rgb/rgb/"

# FTP 연결 설정
ftp = FTP()
ftp.connect(ftp_server, ftp_port)
ftp.login(user=ftp_user, passwd=ftp_password)
ftp.set_pasv(True)

# 동영상 데이터셋 경로
train_json_path = "Diving48_V2_train.json"
test_json_path = "Diving48_V2_test.json"

with open(train_json_path, 'wb') as local_file:
    ftp.retrbinary(f'RETR {"homes/donghuna/database/Diving48_rgb/Diving48_V2_train.json"}', local_file.write)

with open(test_json_path, 'wb') as local_file:
    ftp.retrbinary(f'RETR {"homes/donghuna/database/Diving48_rgb/Diving48_V2_test.json"}', local_file.write)

# 동영상 데이터를 읽어오기 위한 함수
def read_video_from_ftp(ftp, file_path, start_frame, end_frame):
    video_data = io.BytesIO()
    ftp.retrbinary(f'RETR {file_path}', video_data.write)
    video_data.seek(0)
    container = av.open(video_data, format='mp4')
    frames = []
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_frame:
            break
        if i >= start_frame:
            frame_np = frame.to_ndarray(format="rgb24")
            frames.append(frame_np.astype(np.uint8))
    return np.stack(frames, axis=0)

# 변환 함수 정의
def pad_and_resize(frames, target_size=(224, 224), num_frames=100):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    processed_frames = [transform(frame) for frame in frames]

    # Pad or trim the sequence to the desired number of frames
    if len(processed_frames) < num_frames:
        padding = [torch.zeros_like(processed_frames[0]) for _ in range(num_frames - len(processed_frames))]
        processed_frames.extend(padding)
    else:
        processed_frames = processed_frames[:num_frames]

    return torch.stack(processed_frames)

# Diving48 데이터셋 클래스 정의
class Diving48Dataset(Dataset):
    def __init__(self, json_path, ftp, folder_path, target_size=(224, 224), num_frames=20):
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.ftp = ftp
        self.folder_path = folder_path
        self.target_size = target_size
        self.num_frames = num_frames

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        vid_info = self.data[idx]
        vid_name = vid_info['vid_name']
        label = vid_info['label']
        start_frame = vid_info['start_frame']
        end_frame = vid_info['end_frame']
        file_path = os.path.join(self.folder_path, f"{vid_name}.mp4")

        video = read_video_from_ftp(self.ftp, file_path, start_frame, end_frame)
        video = pad_and_resize(video, target_size=self.target_size, num_frames=self.num_frames)
        # video = video.permute(1, 0, 2, 3)  # (T, C, H, W) -> (C, T, H, W)

        return video, label

# 모델 로드
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

# 데이터셋 및 데이터로더 생성
train_dataset = Diving48Dataset(train_json_path, ftp, folder_path)
test_dataset = Diving48Dataset(test_json_path, ftp, folder_path)

def collate_fn(batch):
    videos, labels = zip(*batch)
    videos = torch.stack(videos)
    labels = torch.tensor(labels)
    return videos, labels

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

# 학습 루프 구현
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 손실 함수 정의
loss_fn = torch.nn.CrossEntropyLoss()

# 변환된 동영상 저장 함수
def save_transformed_video(video_tensor, filename):
    # (C, T, H, W) -> (T, C, H, W)
    video_tensor = video_tensor.permute(1, 0, 2, 3)
    for i, frame in enumerate(video_tensor):
        save_image(frame, f"{filename}_frame_{i}.png")

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        videos, labels = batch
        videos = videos.to(device)
        labels = labels.to(device)

        outputs = model(videos)
        loss = loss_fn(outputs.logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 모델 파라미터 저장
    torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pt")

# 평가 루프
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        videos, labels = batch
        videos = videos.to(device)
        labels = labels.to(device)

        outputs = model(videos)
        _, predicted = torch.max(outputs.logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy:.4f}")

# 변환된 동영상 저장 (테스트 데이터셋의 첫 번째 비디오)
video, label = test_dataset[0]
save_transformed_video(video, "transformed_video")

# 연결 종료
ftp.quit()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 